## Get Images marked

### Mark frontal face image automatically.

In [2]:
import os
import numpy as np
import cv2
import sys
import time

sys.path.append('..')
from assist import marker

# marker.frontal_face_marker(r'..\data\00019fa010_940128.tif')
marker.frontal_face_marker(r'..\data\downsyndrome1.png')

array([[180., 136.],
       [183., 163.],
       [188., 188.],
       [193., 214.],
       [201., 240.],
       [214., 264.],
       [233., 285.],
       [255., 301.],
       [281., 306.],
       [310., 302.],
       [339., 288.],
       [366., 270.],
       [386., 247.],
       [400., 221.],
       [409., 192.],
       [416., 163.],
       [420., 132.],
       [191., 128.],
       [204., 115.],
       [224., 113.],
       [243., 117.],
       [263., 123.],
       [297., 121.],
       [319., 114.],
       [341., 109.],
       [364., 113.],
       [382., 126.],
       [279., 144.],
       [278., 157.],
       [277., 170.],
       [276., 184.],
       [262., 203.],
       [269., 205.],
       [277., 206.],
       [286., 205.],
       [296., 203.],
       [217., 145.],
       [229., 137.],
       [244., 137.],
       [255., 151.],
       [243., 156.],
       [227., 155.],
       [313., 151.],
       [324., 137.],
       [340., 137.],
       [353., 145.],
       [342., 154.],
       [326.,

### Change landmarks through dragging (if not satisfied with the auto-marking result).

In [3]:
# marker.mark_modifier(r'..\data\00019fa010_940128.tif')

### Mark profile image manually.

In [4]:
# marker.manual_marker(r'..\data\00019pr010_940128.tif')

## Start 3D fitting

In [13]:
import toml
from core import Blendshape, contour_correspondence, EdgeTopology, fitting, LandmarkMapper, Landmark, MorphableModel, \
    utils, RenderingParameters, render

In [17]:
start_time = time.time()

frontal_pic_name = 'downsyndrome1'
profile_pic_name = 'downsyndrome1'
frontal_img = cv2.imread(os.path.join(r'..\data', frontal_pic_name + '.png'))
profile_img = cv2.imread(os.path.join(r'..\data', profile_pic_name + '.png'))
width = np.shape(frontal_img)[1]
height = np.shape(frontal_img)[0]

s = 2000 / height if height >= width else 2000 / width
scale_param = 900 / height if height >= width else 900 / width

morphable_model = MorphableModel.load_model(r"..\py_share\py_sfm_shape_3448.bin")
blendshapes = Blendshape.load_blendshapes(r"..\py_share\py_expression_blendshapes_3448.bin")
landmark_mapper = LandmarkMapper.LandmarkMapper(r'..\py_share\ibug_to_sfm.txt')
edge_topology = EdgeTopology.load_edge_topology(r'..\py_share\py_sfm_3448_edge_topology.json')
contour_landmarks = contour_correspondence.ContourLandmarks()
contour_landmarks.load(r'..\py_share\ibug_to_sfm.txt')
model_contour = contour_correspondence.ModelContour()
model_contour.load(r'..\py_share\sfm_model_contours.json')
profile_landmark_mapper = LandmarkMapper.ProfileLandmarkMapper(r'..\py_share\profile_to_sfm.txt')

frontal_landmarks = []
landmark_ids = list(map(str, range(1, 69)))  # generates the numbers 1 to 68, as strings
landmarks = utils.read_pts(os.path.join(r'..\data', frontal_pic_name + '.pts'))
for i in range(68):
    frontal_landmarks.append(Landmark.Landmark(landmark_ids[i], [landmarks[i][0] * s, landmarks[i][1] * s]))

profile_landmarks = []
landmarks = utils.read_pts(os.path.join(r'..\data', profile_pic_name + '.pts'))
for x in profile_landmark_mapper.right_mapper.keys():
    coor = landmarks[int(x) - 1]
    profile_landmarks.append(Landmark.Landmark(x, [coor[0] * s, coor[1] * s]))

py_mesh, frontal_rendering_params, profile_rendering_params = fitting.fit_front_and_profile(
    morphable_model, blendshapes, frontal_landmarks, landmark_mapper, profile_landmarks, profile_landmark_mapper,
    round(width * s), round(height * s), edge_topology, contour_landmarks, model_contour, lambda_p=20,
    num_iterations=10)

profile_img = cv2.resize(profile_img, (round(width * scale_param), round(height * scale_param)),
                         interpolation=cv2.INTER_CUBIC)
render.draw_wireframe_with_depth(
    profile_img, py_mesh, profile_rendering_params.get_modelview(), profile_rendering_params.get_projection(),
    RenderingParameters.get_opencv_viewport(width * s, height * s), profile_landmark_mapper, scale_param / s)

frontal_img = cv2.resize(frontal_img, (round(width * scale_param), round(height * scale_param)),
                         interpolation=cv2.INTER_CUBIC)
render.draw_wireframe_with_depth(
    frontal_img, py_mesh, frontal_rendering_params.get_modelview(), frontal_rendering_params.get_projection(),
    RenderingParameters.get_opencv_viewport(width * s, height * s), landmark_mapper, scale_param / s)

for lm in frontal_landmarks:
    cv2.rectangle(
        frontal_img, (int(lm.coordinates[0] * scale_param / s) - 2, int(lm.coordinates[1] * scale_param / s) - 2),
        (int(lm.coordinates[0] * scale_param / s) + 2, int(lm.coordinates[1] * scale_param / s) + 2), (255, 0, 0))

for lm in profile_landmarks:
    cv2.rectangle(
        profile_img, (int(lm.coordinates[0] * scale_param / s) - 2, int(lm.coordinates[1] * scale_param / s) - 2),
        (int(lm.coordinates[0] * scale_param / s) + 2, int(lm.coordinates[1] * scale_param / s) + 2), (255, 0, 0))
    
img = np.hstack([frontal_img, profile_img])

# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

print("--- %s seconds ---" % (time.time() - start_time))

..\core\orthographic_camera_estimation_linear.py:83: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  k = np.linalg.lstsq(a, b)[0]


profile-yaw: -52.512013779756295
profile-pitch: -68.78926301729987
profile-roll: 105.5092249680388
96


..\core\linear_shape_fitting.py:257: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c_s = np.linalg.lstsq(at_omega_reg, rhs)[0]


35.778637912438974
loop 1 is finished at time: 0.27101993560791016
profile-yaw: -52.399488790504684
profile-pitch: -70.32741885957456
profile-roll: 107.14271189942087
11.744294725619401
loop 2 is finished at time: 0.3010225296020508
profile-yaw: -54.98918206753711
profile-pitch: -77.17236248898465
profile-roll: 111.79625025770875
7.897750690188776
loop 3 is finished at time: 0.3460257053375244
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
2.635005993938337
loop 4 is finished at time: 0.38902926445007324
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
1.4026558048804263
loop 5 is finished at time: 0.43203186988830566
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
1.105232951230468
loop 6 is finished at time: 0.4840376377105713
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
0.66469188787

### Load and rescale pictures and landmarks

In [8]:
frontal_pic_name = 'downsyndrome1'
profile_pic_name = 'downsyndrome1'
frontal_img = cv2.imread(os.path.join(r'..\data', frontal_pic_name + '.png'))
profile_img = cv2.imread(os.path.join(r'..\data', profile_pic_name + '.png'))
width = np.shape(frontal_img)[1]
height = np.shape(frontal_img)[0]

s = 2000 / height if height >= width else 2000 / width
scale_param = 900 / height if height >= width else 900 / width

### Load models

In [9]:
morphable_model = MorphableModel.load_model(r"..\py_share\py_sfm_shape_3448.bin")
blendshapes = Blendshape.load_blendshapes(r"..\py_share\py_expression_blendshapes_3448.bin")
landmark_mapper = LandmarkMapper.LandmarkMapper(r'..\py_share\ibug_to_sfm.txt')
edge_topology = EdgeTopology.load_edge_topology(r'..\py_share\py_sfm_3448_edge_topology.json')
contour_landmarks = contour_correspondence.ContourLandmarks()
contour_landmarks.load(r'..\py_share\ibug_to_sfm.txt')
model_contour = contour_correspondence.ModelContour()
model_contour.load(r'..\py_share\sfm_model_contours.json')
profile_landmark_mapper = LandmarkMapper.ProfileLandmarkMapper(r'..\py_share\profile_to_sfm.txt')

frontal_landmarks = []
landmark_ids = list(map(str, range(1, 69)))  # generates the numbers 1 to 68, as strings
landmarks = utils.read_pts(os.path.join(r'..\data', frontal_pic_name + '.pts'))
for i in range(68):
    frontal_landmarks.append(Landmark.Landmark(landmark_ids[i], [landmarks[i][0] * s, landmarks[i][1] * s]))

profile_landmarks = []
landmarks = utils.read_pts(os.path.join(r'..\data', profile_pic_name + '.pts'))
for x in profile_landmark_mapper.right_mapper.keys():
    coor = landmarks[int(x) - 1]
    profile_landmarks.append(Landmark.Landmark(x, [coor[0] * s, coor[1] * s]))

### Do fitting

In [10]:
start_time = time.time()

py_mesh, frontal_rendering_params, profile_rendering_params = fitting.fit_front_and_profile(
    morphable_model, blendshapes, frontal_landmarks, landmark_mapper, profile_landmarks, profile_landmark_mapper,
    round(width * s), round(height * s), edge_topology, contour_landmarks, model_contour, lambda_p=20,
    num_iterations=10)

print("--- %s seconds ---" % (time.time() - start_time))

..\core\orthographic_camera_estimation_linear.py:83: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  k = np.linalg.lstsq(a, b)[0]


profile-yaw: -52.512013779756295
profile-pitch: -68.78926301729987
profile-roll: 105.5092249680388
96


..\core\linear_shape_fitting.py:257: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c_s = np.linalg.lstsq(at_omega_reg, rhs)[0]


35.778637912438974
loop 1 is finished at time: 0.6986696720123291
profile-yaw: -52.399488790504684
profile-pitch: -70.32741885957456
profile-roll: 107.14271189942087
11.744294725619401
loop 2 is finished at time: 0.7266721725463867
profile-yaw: -54.98918206753711
profile-pitch: -77.17236248898465
profile-roll: 111.79625025770875
7.897750690188776
loop 3 is finished at time: 0.7716763019561768
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
2.635005993938337
loop 4 is finished at time: 0.8216793537139893
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
1.4026558048804263
loop 5 is finished at time: 0.8696849346160889
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
1.105232951230468
loop 6 is finished at time: 0.9216902256011963
profile-yaw: -55.8487859049014
profile-pitch: -84.29094520667036
profile-roll: 118.63451534657302
0.66469188787262

### Visualize fitting result

In [11]:
profile_img = cv2.resize(profile_img, (round(width * scale_param), round(height * scale_param)),
                         interpolation=cv2.INTER_CUBIC)
render.draw_wireframe_with_depth(
    profile_img, py_mesh, profile_rendering_params.get_modelview(), profile_rendering_params.get_projection(),
    RenderingParameters.get_opencv_viewport(width * s, height * s), profile_landmark_mapper, scale_param / s)

frontal_img = cv2.resize(frontal_img, (round(width * scale_param), round(height * scale_param)),
                         interpolation=cv2.INTER_CUBIC)
render.draw_wireframe_with_depth(
    frontal_img, py_mesh, frontal_rendering_params.get_modelview(), frontal_rendering_params.get_projection(),
    RenderingParameters.get_opencv_viewport(width * s, height * s), landmark_mapper, scale_param / s)

for lm in frontal_landmarks:
    cv2.rectangle(
        frontal_img, (int(lm.coordinates[0] * scale_param / s) - 2, int(lm.coordinates[1] * scale_param / s) - 2),
        (int(lm.coordinates[0] * scale_param / s) + 2, int(lm.coordinates[1] * scale_param / s) + 2), (255, 0, 0))

for lm in profile_landmarks:
    cv2.rectangle(
        profile_img, (int(lm.coordinates[0] * scale_param / s) - 2, int(lm.coordinates[1] * scale_param / s) - 2),
        (int(lm.coordinates[0] * scale_param / s) + 2, int(lm.coordinates[1] * scale_param / s) + 2), (255, 0, 0))

### Show fitting result

In [12]:
img = np.hstack([frontal_img, profile_img])

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Save result and fitted 3D model.

In [8]:
cv2.imwrite(frontal_pic_name + '-outcome.jpg', img)
render.save_ply(py_mesh, frontal_pic_name + '-output', [210, 183, 108], author='Yinghao Li')

downsyndrome1-output
